In [19]:
import numpy as np
from descent_env import DescentEnv
import random 

In [20]:
# Cambiar render_mode a rgb_array para entrenar/testear
env = DescentEnv(render_mode='rgb_array')

Reading config from C:\Users\fbrio\bluesky\settings.cfg
Reading magnetic variation data
Loading global navigation database...
Reading cache: C:\Users\fbrio\bluesky\cache\navdata.p
Attempt to reimplement AREA from <bound method Area.set_area of <bluesky.plugins.area.Area object at 0x0000022F5E056CE0>> to <bound method Area.set_area of <bluesky.plugins.area.Area object at 0x0000022F5E056CE0>>
Attempt to reimplement EXP from <function init_plugin.<locals>.<lambda> at 0x0000022F5B79C280> to <function init_plugin.<locals>.<lambda> at 0x0000022F5DF9A050>
Attempt to reimplement TAXI from <bound method Area.set_taxi of <bluesky.plugins.area.Area object at 0x0000022F5E056CE0>> to <bound method Area.set_taxi of <bluesky.plugins.area.Area object at 0x0000022F5E056CE0>>
Successfully loaded plugin AREA
Attempt to reimplement DATAFEED from <bound method Modesbeast.toggle of <bluesky.plugins.adsbfeed.Modesbeast object at 0x0000022F5E056C20>> to <bound method Modesbeast.toggle of <bluesky.plugins.adsb

In [21]:
env.observation_space
env.action_space

Box(-1.0, 1.0, (1,), float64)

In [22]:
altitude_space = np.linspace(-30000, 30000, 100)
vertical_velocity_space =  np.linspace(-30000, 30000, 100)
target_altitude_space =  np.linspace(-30000, 30000, 100)
runway_distance_space = np.linspace(-30000, 30000, 100)
altitude_space

array([-30000.        , -29393.93939394, -28787.87878788, -28181.81818182,
       -27575.75757576, -26969.6969697 , -26363.63636364, -25757.57575758,
       -25151.51515152, -24545.45454545, -23939.39393939, -23333.33333333,
       -22727.27272727, -22121.21212121, -21515.15151515, -20909.09090909,
       -20303.03030303, -19696.96969697, -19090.90909091, -18484.84848485,
       -17878.78787879, -17272.72727273, -16666.66666667, -16060.60606061,
       -15454.54545455, -14848.48484848, -14242.42424242, -13636.36363636,
       -13030.3030303 , -12424.24242424, -11818.18181818, -11212.12121212,
       -10606.06060606, -10000.        ,  -9393.93939394,  -8787.87878788,
        -8181.81818182,  -7575.75757576,  -6969.6969697 ,  -6363.63636364,
        -5757.57575758,  -5151.51515152,  -4545.45454545,  -3939.39393939,
        -3333.33333333,  -2727.27272727,  -2121.21212121,  -1515.15151515,
         -909.09090909,   -303.03030303,    303.03030303,    909.09090909,
         1515.15151515,  

In [23]:
def get_state(obs):
    alt = obs['altitude'][0]
    vz = obs['vz'][0]
    target_alt = obs['target_altitude'][0]
    runway_dist = obs['runway_distance'][0]
    alt_idx = min(np.digitize(alt, altitude_space), len(altitude_space)-1)
    vz_idx = min(np.digitize(vz, vertical_velocity_space), len(vertical_velocity_space)-1)
    target_alt_idx = min(np.digitize(target_alt, target_altitude_space), len(target_altitude_space)-1)
    runway_dist_idx = min(np.digitize(runway_dist, runway_distance_space), len(runway_distance_space)-1)
    return alt_idx, vz_idx, target_alt_idx, runway_dist_idx

In [24]:
obs = env.observation_space.sample()
print(obs)
state = get_state(obs) # Ejemplo de obs
state

OrderedDict([('altitude', array([0.05850388])), ('runway_distance', array([-0.3033643])), ('target_altitude', array([0.66270483])), ('vz', array([-1.00262884]))])


(50, 50, 50, 50)

In [25]:
actions = list(np.linspace(-1, 1, 10))
actions

[-1.0,
 -0.7777777777777778,
 -0.5555555555555556,
 -0.33333333333333337,
 -0.11111111111111116,
 0.11111111111111116,
 0.33333333333333326,
 0.5555555555555554,
 0.7777777777777777,
 1.0]

In [26]:
def get_sample_action():
    return random.choice(actions)

In [27]:
Q = np.zeros((len(altitude_space), len(vertical_velocity_space), len(target_altitude_space), len(runway_distance_space), len(actions)))
Q

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0.

In [28]:
# Tamaños de cada dimensión del estado
n_alt = len(altitude_space)
n_vz = len(vertical_velocity_space)
n_target_alt = len(target_altitude_space)
n_rwy_dist = len(runway_distance_space)
n_actions = len(actions)

# Inicializamos Q-table con ceros
Q = np.zeros((n_alt, n_vz, n_target_alt, n_rwy_dist, n_actions))


In [29]:
def choose_action(state, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(n_actions)  # acción aleatoria
    else:
        return np.argmax(Q[state])  # mejor acción según Q


In [30]:
# Hiperparámetros
alphas = [0.8] # Tasa de aprendizaje
gammas = [0.7] # Factor de descuento
episodes = 5000 # Episodios de entrenamiento
test_episodes = 100  # Episodios de testeo

epsilon = 1  # Valor inicial de epsilon
epsilon_decay = 0.999  # Decaimiento de epsilon
epsilon_min = 0.01  # Valor mínimo de epsilon

In [31]:
results = []

In [32]:
def train_stochastic(alpha, gamma, epsilon, epsilon_min, epsilon_decay, episodes):
    Q = np.zeros((n_alt, n_vz, n_target_alt, n_rwy_dist, n_actions))
    rewards_per_episode = []

    for ep in range(episodes):
        obs, _ = env.reset()
        state = get_state(obs)
        done = False
        total_reward = 0

        while not done:
            action_idx = choose_action(state, epsilon)
            action = np.array([actions[action_idx]])

            next_obs, reward, terminated, truncated, _ = env.step(action)
            next_state = get_state(next_obs)
            done = terminated or truncated

            # Stochastic Q-learning: sample next action for update
            next_action_idx = np.random.randint(n_actions)
            td_target = reward + gamma * Q[next_state + (next_action_idx,)]
            td_error = td_target - Q[state + (action_idx,)]
            Q[state + (action_idx,)] += alpha * td_error

            state = next_state
            total_reward += reward

        rewards_per_episode.append(total_reward)
        epsilon = max(epsilon * epsilon_decay, epsilon_min)

    return Q, np.mean(rewards_per_episode)

In [33]:
def test_agent(Q):
    rewards = []

    for _ in range(test_episodes):
        obs, _ = env.reset()
        state = get_state(obs)
        done = False
        total_reward = 0

        while not done:
            action_idx = np.argmax(Q[state])  # greedy
            action = np.array([actions[action_idx]])
            next_obs, reward, terminated, _, _ = env.step(action)
            state = get_state(next_obs)
            done = terminated
            total_reward += reward

        rewards.append(total_reward)

    return np.mean(rewards)

In [34]:
# Entrenamiento y testing usando Q-learning estocástico
print(f"\n=== Entrenando (Stochastic Q-learning) con epsilon: {epsilon:.2f} ===")
print(f"\n--- Entrenando con {episodes} episodios ---")
for alpha in alphas:
    for gamma in gammas:
        print(f"\n--- Entrenando con alpha: {alpha:.2f}, gamma: {gamma:.2f} ---")
        Q_stoch, train_avg_reward_stoch = train_stochastic(alpha, gamma, epsilon, epsilon_min, epsilon_decay, episodes)
        print(f"--- Entrenamiento estocástico finalizado con epsilon {epsilon:.2f} ---")    
        print(f"--- Evaluando desempeño estocástico (testeo sin exploración) ---")
        test_avg_reward_stoch = test_agent(Q_stoch)
        results.append({
            "alpha": alpha,
            "gamma": gamma,
            "episodes": episodes, 
            "train_avg_reward": train_avg_reward_stoch,
            "test_avg_reward": test_avg_reward_stoch,
            "stochastic": True
        })
        with open("resultados_qlearning.txt", "w") as f:
            for res in results:
                f.write(str(res) + "\n")
                print(f"Resultados: {res}")


=== Entrenando (Stochastic Q-learning) con epsilon: 1.00 ===

--- Entrenando con 5000 episodios ---

--- Entrenando con alpha: 0.80, gamma: 0.70 ---
--- Entrenamiento estocástico finalizado con epsilon 1.00 ---
--- Evaluando desempeño estocástico (testeo sin exploración) ---
Resultados: {'alpha': 0.8, 'gamma': 0.7, 'episodes': 5000, 'train_avg_reward': -125.25518649974167, 'test_avg_reward': -127.64329473333332, 'stochastic': True}
